In [1]:
# Example on how to use PopulationData class for preprocessing
import glob
import os
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm
import json
from pathlib import Path
from typing import Dict, List
import h5py
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import pdist
from ephysvibe.structures.neuron_data import NeuronData
from ephysvibe.trials.spikes import firing_rate
from ephysvibe.trials import align_trials, select_trials
from ephysvibe.structures.population_data import PopulationData
from datetime import datetime
import pandas as pd
seed = 1997

In [3]:
# Define parameters
areas = ["lip"]
subject = "Riesling"
avgwin = 100
min_sp_sec = 1
n_test = 1
min_trials = 25
nonmatch = True  # if True: includes nonmatch trials
norm = False
zscore = True
# sample timing
time_before_sample = 500
start_sample = -200
end_sample = 450 + 400
# test timing
time_before_test = 500
start_test = -400
end_test = n_test * 450 + 200
# paths
filepaths = {
    "lip": "D:/data/session_struct/lip/neurons/",
    "pfc": "/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/new_structure/session_struct/pfc/neurons/",
    "v4": "/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/new_structure/session_struct/v4/neurons/",
}
# -------------------------------------------- End parameters ------------------------------------------
# Compute idxs
idx_start_sample = time_before_sample + start_sample
idx_end_sample = time_before_sample + end_sample
idx_start_test = time_before_test + start_test
idx_end_test = time_before_test + end_test
# total trial duration
trial_dur = end_sample - start_sample + end_test - start_test

In [4]:
for area in areas:
    path = filepaths[area]
    neu_path = path + "*neu.h5"
    path_list = glob.glob(neu_path)

In [5]:
attr_dtype = {'sp_samples':np.float16,'cluster_ch':np.float16,'cluster_number':np.float16,'block':np.float16,'trial_error':np.float16,'sample_id':np.float16,
              'test_stimuli':np.float16,'test_distractor':np.float16,'cluster_id':np.float16,'cluster_depth':np.float16,'code_samples':np.float16,
              'position':np.float16,'pos_code':np.float16,'code_numbers':np.float16}
popu=PopulationData.get_population(path_list[  5:15], attr_dtype)

100%|██████████| 10/10 [00:00<00:00, 111.12it/s]


In [6]:
popu.to_python_hdf5('kk.h5')

In [5]:
popu=PopulationData.from_python_hdf5('kk.h5')

In [9]:
popu.population[9].sp_samples

array([[ 0.,  0.,  0., ..., nan, nan, nan],
       [ 0.,  0.,  0., ..., nan, nan, nan],
       [ 0.,  0.,  0., ..., nan, nan, nan],
       ...,
       [ 0.,  0.,  0., ..., nan, nan, nan],
       [ 0.,  0.,  0., ..., nan, nan, nan],
       [ 0.,  0.,  0., ..., nan, nan, nan]], dtype=float16)

In [6]:
df = popu.execute_function(NeuronData.get_sp_per_sec)

100%|██████████| 10/10 [00:00<00:00, 3009.91it/s]


In [6]:
df=pd.read_csv('C:/Users/camil/Documents/int/code/Users/losadac/preproc/testdf.csv')

In [9]:
mean_fr = (df['in_mean_fr_sample']+df['in_mean_fr_delay'])/2
sel_neu = df[mean_fr>1]['nid'].values

In [10]:
popu.population[0].sp_test

AttributeError: 'NeuronData' object has no attribute 'sp_test'

In [11]:
subpopu = popu.get_subpopulation(sel_neu)

In [12]:
subpopu.population[0].sp_test

AttributeError: 'list' object has no attribute 'population'

In [16]:
def get_sample_test(neu:NeuronData,params:Dict):
    time_before_sample=params['time_before_sample']
    time_before_sample=params['time_before_test']
    sp_sample, mask = neu.align_on(
        select_block=params['select_block'],
        select_pos=params['select_pos'],
        event="sample_on",
        time_before=params['time_before_sample'],
        error_type=0,
    )
    sp_test, mask = neu.align_on(
        select_block=params['select_block'],
        select_pos=params['select_pos'],
        event="test_on_1",
        time_before=params['time_before_test'],
        error_type=0,
    )
    idx_start_sample = time_before_sample + params['start_sample']
    idx_end_sample = time_before_sample + params['end_sample']
    idx_start_test = time_before_test + params['start_test']
    idx_end_test = time_before_test + params['end_test']

    new_values = {}
    new_values['sp_sample'] = np.array(sp_sample[:,idx_start_sample:idx_end_sample],dtype=np.int8)
    new_values['sp_test'] = np.array(sp_test[:,idx_start_test:idx_end_test],dtype=np.int8)
    
    for attr_name in [
        "block",
        "trial_error",
        "code_samples",
        "code_numbers",
        "position",
        "pos_code",
        "sample_id",
        "test_stimuli",
        "test_distractor",
    ]:
        new_values[attr_name] = getattr(neu, attr_name)[mask]
    neu.edit_attributes(new_values)

In [17]:
params = {
    # sample
    'time_before_sample': 500,
    'start_sample': -200,
    'end_sample': 450 + 400,
    # test
    'time_before_test': 500,
    'start_test': -400,
    'end_test': n_test * 450 + 200,
    'select_block':1,
    'select_pos':1,
    'event':"sample_on",
    'error_type':0,
}

In [13]:
subpopu = PopulationData(subpopu)

In [18]:
_=subpopu.execute_function(get_sample_test,params=params,n_jobs=1,ret_df=False)

100%|██████████| 4/4 [00:00<00:00, 12.10it/s]


In [19]:
subpopu.population[0].sp_sample.shape

(265, 1050)

In [7]:
df.head()

,nid,in_mean_fr_sample_NN,in_lat_max_fr_sample_NN,in_mean_max_fr_sample_NN,in_mean_fr_delay_NN,in_lat_max_fr_delay_NN,in_mean_max_fr_delay_NN,in_mean_fr_sample_N,in_lat_max_fr_sample_N,in_mean_max_fr_sample_N,...,out_mean_max_fr_sample_NN,out_mean_fr_delay_NN,out_lat_max_fr_delay_NN,out_mean_max_fr_delay_NN,out_mean_fr_sample_N,out_lat_max_fr_sample_N,out_mean_max_fr_sample_N,out_mean_fr_delay_N,out_lat_max_fr_delay_N,out_mean_max_fr_delay_N
0,good1LIP2023-10-18_10-57-56Riesling,6.492367,136,8.078053,1.131511,3,1.642847,5.684495,139,8.851400,...,7.281459,1.614321,4,1.671470,4.872106,70.0,6.508536,2.933936,129.0,3.224715
1,mua8LIP2023-10-18_10-57-56Riesling,1.017039,42,1.766470,2.805549,234,3.710537,1.833958,273,2.898548,...,2.087411,3.820307,321,4.744953,0.946610,349.0,1.440784,1.426995,335.0,2.209467
2,mua9LIP2023-10-18_10-57-56Riesling,0.491892,380,0.808868,1.021345,221,1.341812,0.692484,427,0.779571,...,1.006486,0.573862,176,0.592177,0.894014,103.0,1.303982,0.546876,0.0,0.750160
3,mua10LIP2023-10-18_10-57-56Riesling,1.563345,108,3.349809,0.494177,1,0.728646,1.973887,94,2.795180,...,0.817303,0.605666,37,0.906040,0.897311,394.0,1.148393,1.451206,379.0,2.199128
4,mua11LIP2023-10-18_10-57-56Riesling,2.032776,100,4.071189,1.638862,155,1.862226,1.083320,92,2.041631,...,1.619150,0.854565,392,1.025324,0.681207,384.0,1.096511,1.508641,271.0,1.896699


: 

In [ ]:
def get_matrix_position(neu_data,matrix_df,ch_start):
    ch_start = np.load(
        sessions_path
        + session
        + "/Record Node 102/experiment1/recording1/continuous/Acquisition_Board-100.Rhythm Data/KS"
        + area.upper()
        + "/channel_map.npy"
    )[0][0]
    cluster_ch = neu_data.cluster_ch - ch_start
    matrix_df = pd.read_csv(
        ch_pos_path + area + "_ch_pos.csv",
        header=0,
        index_col=0,
    )
    matrix = matrix_df.values
    matrix = matrix - matrix.min().min()
    row, col = np.where(cluster_ch == matrix)
    return row,col